[TOC]

## 随机森林算法梳理

1. 集成学习的概念
2. 个体学习器的概念
3. boosting bagging的概念、异同点
4. 理解不同的结合策略(平均法，投票法，学习法)
5. 随机森林的思想
6. 随机森林的推广
7. 随机森林的优缺点
8. 随机森林在sklearn中的参数解释
9. 随机森林的应用场景

### 1. 集成学习的概念

集成学习(ensemble learning)通过构建并结合多个学习器来完成任务。

### 2. 个体学习器的概念

个体学习器通常由一个现有的学习算法从训练数据中产生，例如C4.5，BP神经网络算法等，此时集成算法中只包含同种类型的个体学习器，例如决策树集成中全是决策树，“神经网络集成”中全是神经网络，这样的集成是同质的。同质集成中的个体学习器亦称为“基学习器”（base learner），相应的学习算法亦称为“基学习算法”（base learning algorithm）。集成也可包含不同类型的个体学习器这时不再有基学习算法；相应的，个体学习器一般不称为基学习器，常称为“组件学习器”或直接称为个体学习器。

### 3. boosting bagging的概念、异同点

根据个体学习器的生成方式，目前的集成学习方法大致可分为两大类，分别为Boosting和Bagging。

- Bagging: bootstrap aggregation ，其实就是并行训练一堆分类器，取平均

  - Bagging的代表RL：每棵树有放回的随机选择数据且有放回的随机选择特征，生成不同的树；保证每颗树数据和特征的量是一样的，保证了泛化能力
  - RL优点：
    - 他能处理很高维度的数据，并且不用做特征选择
    - 在训练之后，它能给出哪些特征比较重要。控制变量法选出影响大的特征
    - 容易做出并行化方法，速度快
    - 树的优点，可以进行可视化分析

- Boosting: 从弱分类器开始串行训练，加强模型，通过加权进行训练

  - AdaBoost：根据前一次的分类效果调整权重，重视出现误差的样本

  $$
  F_{m}(x)=F_{m-1}(x) + argmin_{h}\sum_{i=1}^{n}L(y_{i},F_{m-1}(x_i)+h(x_i))
  $$

### 4.理解不同的结合策略(平均法，投票法，学习法)

#### 平均法

- 对数值型输出$h_i (X) εI$， 最常见的结合策略是使用平均法(averaging).
  - 简单平均法(simple averaging)：简单的累加求均值的额方法
  - 加权平均法(weighted averaging)：为每个个体学习器分配权重累加，求和，求平均的方法
  - 显然，简单平均法是加权平均法$w_i = l/T$ 的特例.

#### 投票法

- 对分类任务来说，学习器$h_i$ 将从类别标记集合${C1 ，C2，... ， CN}$ 中预测出一个标记?最常见的结合策略是使用投票法(voting).
  - 绝对多数投票法：即若某标记得票过半数，则预测为该标记;否则拒绝预测.
  - 相对多数投票法(plurality voting)：即预测为得票最多的标记，若同时有多个标记获最高票，则从中随机选取一个。
  - 加权投票法：与加权平均法类似， $W_i$ 是$h_i$的权重，通常$w_i$>0，累加和=1.
- 简单来说，投票法就是少数服从多数

#### 学习法

- 当训练数据很多时，一种更为强大的结合策略是使用“学习法”，即通过另一个学习器来进行结合。stacking是学习法的典型代表。
- stacking本身是一种著名的集成学习方法，且有不少集成学习算法可视为其变体或特例。一般的资料介绍集成算法有三种，分别为boosting、bagging和stacking。西瓜书上将stacking算法放在此处进行介绍。
- Stacking先从初始数据集训练出初级学习器，然后"生成"一个新数据集用于训练次级学习器.在这个新数据集中，初级学习器的输出被当作样例输入特征，而初始样本的y仍被当作样例y

### 5. 随机森林的思想

随机森林(Randon Forest,简称RF)是Bagging的一个扩展变体。RF在决策树为基学习器构建Bagging集成的基础上，进一步在决策树的训练过程中
引入随机属性选择。具体来讲，传统决策树在选择划分属性时是在当前结点的属性集合中选择一个最优属性；而在RF中，对基决策树的每个结点，
先从该结点的属性集合中随机选择一个包含k个属性的子集，然后再从这个子集中选择最优属性用于划分

### 6. 随机森林的推广

由于RF在实际应用中的良好特性，基于RF，有很多变种算法，应用也很广泛，不光可以用于分类回归，还可以用于特征转换，异常点检测等。下面对于这些RF家族的算法中有代表性的做一个总结。

#### extra trees

　　　　extra trees是RF的一个变种, 原理几乎和RF一模一样，仅有区别有：

　　　　1） 对于每个决策树的训练集，RF采用的是随机采样bootstrap来选择采样集作为每个决策树的训练集，而extra trees一般不采用随机采样，即每个决策树采用原始训练集。

　　　　2） 在选定了划分特征后，RF的决策树会基于基尼系数，均方差之类的原则，选择一个最优的特征值划分点，这和传统的决策树相同。但是extra trees比较的激进，他会随机的选择一个特征值来划分决策树。

　　　　从第二点可以看出，由于随机选择了特征值的划分点位，而不是最优点位，这样会导致生成的决策树的规模一般会大于RF所生成的决策树。也就是说，模型的方差相对于RF进一步减少，但是偏倚相对于RF进一步增大。在某些时候，extra trees的泛化能力比RF更好。

#### Totally Random Trees Embedding

　　　　Totally Random Trees Embedding(以下简称 TRTE)是一种非监督学习的数据转化方法。它将低维的数据集映射到高维，从而让映射到高维的数据更好的运用于分类回归模型。我们知道，在支持向量机中运用了核方法来将低维的数据集映射到高维，此处TRTE提供了另外一种方法。

　　　　TRTE在数据转化的过程也使用了类似于RF的方法，建立T个决策树来拟合数据。当决策树建立完毕以后，数据集里的每个数据在T个决策树中叶子节点的位置也定下来了。比如我们有3颗决策树，每个决策树有5个叶子节点，某个数据特征xx划分到第一个决策树的第2个叶子节点，第二个决策树的第3个叶子节点，第三个决策树的第5个叶子节点。则x映射后的特征编码为(0,1,0,0,0,     0,0,1,0,0,     0,0,0,0,1), 有15维的高维特征。这里特征维度之间加上空格是为了强调三颗决策树各自的子编码。

　　　　映射到高维特征后，可以继续使用监督学习的各种分类回归算法了。

#### Isolation Forest

　　　　Isolation Forest（以下简称IForest）是一种异常点检测的方法。它也使用了类似于RF的方法来检测异常点。

　　　　对于在T个决策树的样本集，IForest也会对训练集进行随机采样,但是采样个数不需要和RF一样，对于RF，需要采样到采样集样本个数等于训练集个数。但是IForest不需要采样这么多，一般来说，采样个数要远远小于训练集个数？为什么呢？因为我们的目的是异常点检测，只需要部分的样本我们一般就可以将异常点区别出来了。

　　　　对于每一个决策树的建立， IForest采用随机选择一个划分特征，对划分特征随机选择一个划分阈值。这点也和RF不同。

　　　　另外，IForest一般会选择一个比较小的最大决策树深度max_depth,原因同样本采集，用少量的异常点检测一般不需要这么大规模的决策树。

　　　　对于异常点的判断，则是将测试样本点xx拟合到T颗决策树。计算在每颗决策树上该样本的叶子节点的深度
$$
h_x(x)
$$
。从而可以计算出平均高度h(x)。此时我们用下面的公式计算样本点xx的异常概率:
$$
s(x,m)=2^{-\frac{h(m)}{c(m)}}
$$


　　　　其中，m为样本个数。

$c(m)$的表达式为：
$$
c(m)=2ln(m-1)+ \xi-2\frac {m-1} {m}
$$
为欧拉常数



　　　　s(x,m)的取值范围是[0,1],取值越接近于1，则是异常点的概率也越大。

### 7.随机森林的优缺点

- 优点
  - 训练可以高度并行化，对于大数据时代的大样本训练速度有优势。个人觉得这是的最主要的优点。
  - 由于可以随机选择决策树节点划分特征，这样在样本特征维度很高的时候，仍然能高效的训练模型。
  - 在训练后，可以给出各个特征对于输出的重要性
  - 由于采用了随机采样，训练出的模型的方差小，泛化能力强。
  - 相对于Boosting系列的Adaboost和GBDT， RF实现比较简单。
  - 对部分特征缺失不敏感。
- 缺点
  - 在某些噪音比较大的样本集上，RF模型容易陷入过拟合。
  - 取值划分比较多的特征容易对RF的决策产生更大的影响，从而影响拟合的模型的效果。

### ８.随机森林在sklearn中的参数解释

sklearn.ensemble.RandomForestClassifier

class sklearn.ensemble.RandomForestClassifier(n_estimators=’warn’, criterion=’gini’, max_depth=None,

min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’,

max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True,

oob_score=False, n_jobs=None, random_state=None, verbose=0,

warm_start=False, class_weight=None)[source]

- `n_estimators`:森林中树的数量，初始越多越好，但是会增加训练时间，到达一定数量后模型的表现不会再有显著的提升
- `criterion`:特征选择的标准，有信息增益和基尼系数两种，使用信息增益的是ID3和C4.5算法（使用信息增益比），使用基尼系数的CART算法，默认是gini系数。
- `max_depth`:决策树最大深度，决策树模型先对所有数据集进行切分，再在子数据集上继续循环这个切分过程，max_depth可以理解成用来限制这个循环次数。
- `min_samples_split`:子数据集再切分需要的最小样本量，默认是2，如果子数据样本量小于2时，则不再进行下一步切分。如果数据量较小，使用默认值就可，如果数据量较大，为降低计算量，应该把这个值增大，即限制子数据集的切分次数。
- `min_samples_leaf=1`:叶节点（子数据集）最小样本数，如果子数据集中的样本数小于这个值，那么该叶节点和其兄弟节点都会被剪枝（去掉），该值默认为1。
- `min_weight_fraction_leaf`:在叶节点处的所有输入样本权重总和的最小加权分数，如果不输入则表示所有的叶节点的权重是一致的。
- `max_features`:特征切分时考虑的最大特征数量，默认是对所有特征进行切分，也可以传入int类型的值，表示具体的特征个数；也可以是浮点数，则表示特征个数的百分比；还可以是sqrt,表示总特征数的平方根；也可以是log2，表示总特征数的log个特征。
- `max_leaf_nodes`:最大叶节点个数，即数据集切分成子数据集的最大个数。
- `min_impurity_decrease`:切分点不纯度最小减少程度，如果某个结点的不纯度减少小于这个值，那么该切分点就会被移除。
- `min_impurity_split`:切分点最小不纯度，用来限制数据集的继续切分（决策树的生成），如果某个节点的不纯度（可以理解为分类错误率）小于这个阈值，那么该点的数据将不再进行切分。
- `bootstrap=True`:bootstrap采样，默认为True
- `oob_score=False`:oob（out of band，带外）数据，即：在某次决策树训练中没有被bootstrap选中的数据。多单个模型的参数训练，我们知道可以用cross validation（cv）来进行，但是特别消耗时间，而且对于随机森林这种情况也没有大的必要，所以就用这个数据对决策树模型进行验证，算是一个简单的交叉验证。性能消耗小，但是效果不错。
- `n_jobs=None`:n_jobs,并行化，可以在机器的多个核上并行的构造树以及计算预测值，不过受限于通信成本，可能效率并不会说分为k个线程就得到k倍的提升，不过整体而言相对需要构造大量的树或者构建一棵复杂的树而言还是高效的.
- `random_state=None`:随机数种子，类似于train_test_split分割时所使用的random_state
- `verbose=0`:是否显示任务进程。
- `class_weight`:权重设置，主要是用于处理不平衡样本，与LR模型中的参数一致，可以自定义类别权重，也可以直接使用balanced参数值进行不平衡样本处理。

### 9.随机森林的应用场景

随机森林应用很广泛，分类和回归问题都可以解决；甚至可以用来进行特征选择，因为随机森林有一个很重要的参数，叫做特征重要性，可以帮助我们筛选出重要的特征。在不知道使用什么算法的时候，可以使用随机森林来进行建模求解。